In [1]:
# import neccessary dependencies
import numpy as np
from numpy import array
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import svm
from sklearn.metrics import accuracy_score

# Import the data

In [2]:
# training data
train = pd.read_csv(r"C:\Users\Admin\OneDrive\Documents\Webdev\hatespeech\train.csv", nrows=5000)

# testing data
test = pd.read_csv(r"C:\Users\Admin\OneDrive\Documents\Webdev\hatespeech\test.csv", nrows=5000)


# #training data
# train = pd.read_csv(r"C:\Users\jmest\Documents\Files\C4S2\Thesis\ThesisFiles-main\ThesisFiles-main\hatespeech\test.csv", nrows=5000)

# #testing data
# test = pd.read_csv(r"C:\Users\jmest\Documents\Files\C4S2\Thesis\ThesisFiles-main\ThesisFiles-main\hatespeech\test.csv", nrows=5000)




# Clean the data of null values

In [3]:
# drop rows with null values
train.dropna()

,text,label
0,Inaasahan na ni Vice President Jejomar Binay n...,0
1,Mar Roxas TANG INA TUWID NA DAAN DAW .. EH SYA...,1
2,Salamat sa walang sawang suporta ng mga taga m...,0
3,@rapplerdotcom putangina mo binay TAKBO PA,1
4,"Binay with selective amnesia, forgetting about...",0
...,...,...
4995,Awww DUTERTE Na wag Lang si Roxas,0
4996,RT @mikkieugenio: If the SC disqualifies Poe a...,0
4997,"""Pag naging presidente si Binay, wala kayong t...",0
4998,Yan na naman ang walang kwentang commercial ni...,1


In [4]:
#check for null values in train
train.isnull().sum()

text     0
label    0
dtype: int64

In [5]:
# check for 0 values in train
sum(train["label"] == 0)

2653

In [6]:
# check for 1 values in train
sum(train["label"] == 1)

2347

In [7]:
# drop null values in test
test.dropna()

,text,label
0,Unshaded votes and votes for Mayor Duterte goe...,1
1,#NoMoreChance https://t.co/msaaUGv0bS,1
2,@itsmanj well there's other good choices like ...,0
3,Nognog. Pandak. Laki sa hirap. Pero corrupt. Y...,1
4,Ex-Binay aide turns tables on Mercado | https:...,0
...,...,...
4227,@Ohohnikk @raachelleele kay nancy binay gagand...,1
4228,RT @chin_tessane: @ImangKuba @Senyora hahahaha...,1
4229,#onlybinay,0
4230,Trend Alert: 'Abby Binay'. More trends at http...,0


In [8]:
test.isnull().sum()

text     0
label    0
dtype: int64

In [9]:
train_shape = train.shape
print(train_shape)

# 10000 train data is available

(5000, 2)


In [10]:
test_shape = test.shape
print(test_shape)

#4232 test data is available

(4232, 2)


# Clean the data of unwated Text and Characters

In [11]:
# remove special characters using the regular expression library

import re

#set up punctuations we want to be replaced

REPLACE_NO_SPACE = re.compile("(\.)|(\;)|(\:)|(\!)|(\')|(\?)|(\,)|(\")|(\|)|(\()|(\))|(\[)|(\])|(\%)|(\$)|(\>)|(\<)|(\{)|(\})")
REPLACE_WITH_SPACE = re.compile("(<br\s/><br\s/?)|(-)|(/)|(:).")

In [12]:
import preprocessor as p

# custum function to clean the dataset (combining tweet_preprocessor and reguar expression)
def clean_tweets(df):
  tempArr = []
  for line in df:
    # send to tweet_processor
    tmpL = p.clean(line)
    # remove puctuation
    tmpL = REPLACE_NO_SPACE.sub("", tmpL.lower()) # convert all tweets to lower cases
    tmpL = REPLACE_WITH_SPACE.sub(" ", tmpL)
    tempArr.append(tmpL)
  return tempArr

In [13]:
# clean training data
train_tweet = clean_tweets(train["text"])
train_tweet = pd.DataFrame(train_tweet)

In [14]:
# append cleaned tweets to the training data
train["clean_tweet"] = train_tweet

# compare the cleaned and uncleaned tweets
train.head(10)

,text,label,clean_tweet
0,Inaasahan na ni Vice President Jejomar Binay n...,0,inaasahan na ni vice president jejomar binay n...
1,Mar Roxas TANG INA TUWID NA DAAN DAW .. EH SYA...,1,mar roxas tang ina tuwid na daan daw eh sya n...
2,Salamat sa walang sawang suporta ng mga taga m...,0,salamat sa walang sawang suporta ng mga taga m...
3,@rapplerdotcom putangina mo binay TAKBO PA,1,putangina mo binay takbo pa
4,"Binay with selective amnesia, forgetting about...",0,binay with selective amnesia forgetting about ...
5,It doesn't matter whoever won between Duterte ...,0,it doesnt matter whoever won between duterte &...
6,Nognog? Pero nognog din ang nag malasakit? Wtf...,1,nognog pero nognog din ang nag malasakit wtf t...
7,#OnlyB1nay ?? #FB https://t.co/QEQnsK67Gm,1,
8,What Abi Binay said on running for Makati mayo...,0,what abi binay said on running for makati mayor
9,Srsly. How can Binay do away with no tax for t...,1,srsly how can binay do away with no tax for th...


In [15]:
sum(train['clean_tweet'] == '')

253

In [16]:
#df['Another'] = df['Another'].replace('', np.nan)
#replace all empty spaces with NaN to drop using dropna
train['clean_tweet'] = train['clean_tweet'].replace('', np.NaN)

In [17]:
train.dropna(axis='rows')

,text,label,clean_tweet
0,Inaasahan na ni Vice President Jejomar Binay n...,0,inaasahan na ni vice president jejomar binay n...
1,Mar Roxas TANG INA TUWID NA DAAN DAW .. EH SYA...,1,mar roxas tang ina tuwid na daan daw eh sya n...
2,Salamat sa walang sawang suporta ng mga taga m...,0,salamat sa walang sawang suporta ng mga taga m...
3,@rapplerdotcom putangina mo binay TAKBO PA,1,putangina mo binay takbo pa
4,"Binay with selective amnesia, forgetting about...",0,binay with selective amnesia forgetting about ...
...,...,...,...
4994,@jefk_rew pity those poor people who were invo...,1,pity those poor people who were involved in th...
4995,Awww DUTERTE Na wag Lang si Roxas,0,awww duterte na wag lang si roxas
4996,RT @mikkieugenio: If the SC disqualifies Poe a...,0,if the sc disqualifies poe as president eithe...
4997,"""Pag naging presidente si Binay, wala kayong t...",0,pag naging presidente si binay wala kayong tax...


In [18]:
train.head(10)

,text,label,clean_tweet
0,Inaasahan na ni Vice President Jejomar Binay n...,0,inaasahan na ni vice president jejomar binay n...
1,Mar Roxas TANG INA TUWID NA DAAN DAW .. EH SYA...,1,mar roxas tang ina tuwid na daan daw eh sya n...
2,Salamat sa walang sawang suporta ng mga taga m...,0,salamat sa walang sawang suporta ng mga taga m...
3,@rapplerdotcom putangina mo binay TAKBO PA,1,putangina mo binay takbo pa
4,"Binay with selective amnesia, forgetting about...",0,binay with selective amnesia forgetting about ...
5,It doesn't matter whoever won between Duterte ...,0,it doesnt matter whoever won between duterte &...
6,Nognog? Pero nognog din ang nag malasakit? Wtf...,1,nognog pero nognog din ang nag malasakit wtf t...
7,#OnlyB1nay ?? #FB https://t.co/QEQnsK67Gm,1,NaN
8,What Abi Binay said on running for Makati mayo...,0,what abi binay said on running for makati mayor
9,Srsly. How can Binay do away with no tax for t...,1,srsly how can binay do away with no tax for th...


In [19]:
train.tail(10)

,text,label,clean_tweet
4990,Bank robber turned multi-millionaire arrested ...,0,bank robber turned multi millionaire arrested ...
4991,Nakakairita yung tvc na para kay binay. Hehe,1,nakakairita yung tvc na para kay binay hehe
4992,SOBRA ?????? Puro sila Roxas samantalang pag k...,1,sobra puro sila roxas samantalang pag kay dut...
4993,ElectionHugot:“We lose the presidency not once...,0,electionhugotwe lose the presidency not once b...
4994,@jefk_rew pity those poor people who were invo...,1,pity those poor people who were involved in th...
4995,Awww DUTERTE Na wag Lang si Roxas,0,awww duterte na wag lang si roxas
4996,RT @mikkieugenio: If the SC disqualifies Poe a...,0,if the sc disqualifies poe as president eithe...
4997,"""Pag naging presidente si Binay, wala kayong t...",0,pag naging presidente si binay wala kayong tax...
4998,Yan na naman ang walang kwentang commercial ni...,1,yan na naman ang walang kwentang commercial ni...
4999,https://t.co/CeNBKhrm8P,0,NaN


In [20]:
#https://statisticsglobe.com/drop-rows-blank-values-from-pandas-dataframe-python
train['clean_tweet'] = train['clean_tweet'].replace('', float('NaN'), regex = True)

In [21]:
train.dropna(inplace= True)

In [22]:
train.head(10)

,text,label,clean_tweet
0,Inaasahan na ni Vice President Jejomar Binay n...,0,inaasahan na ni vice president jejomar binay n...
1,Mar Roxas TANG INA TUWID NA DAAN DAW .. EH SYA...,1,mar roxas tang ina tuwid na daan daw eh sya n...
2,Salamat sa walang sawang suporta ng mga taga m...,0,salamat sa walang sawang suporta ng mga taga m...
3,@rapplerdotcom putangina mo binay TAKBO PA,1,putangina mo binay takbo pa
4,"Binay with selective amnesia, forgetting about...",0,binay with selective amnesia forgetting about ...
5,It doesn't matter whoever won between Duterte ...,0,it doesnt matter whoever won between duterte &...
6,Nognog? Pero nognog din ang nag malasakit? Wtf...,1,nognog pero nognog din ang nag malasakit wtf t...
8,What Abi Binay said on running for Makati mayo...,0,what abi binay said on running for makati mayor
9,Srsly. How can Binay do away with no tax for t...,1,srsly how can binay do away with no tax for th...
10,"Sen Grace Poe, puro ka puso. Kaya lugmok bansa...",1,sen grace poe puro ka puso kaya lugmok bansa n...


In [23]:
train.tail(10)

,text,label,clean_tweet
4989,"Mar Roxas on MRT ""Yan na ang nadatnan natin."" ...",1,mar roxas on mrt yan na ang nadatnan natin ay ...
4990,Bank robber turned multi-millionaire arrested ...,0,bank robber turned multi millionaire arrested ...
4991,Nakakairita yung tvc na para kay binay. Hehe,1,nakakairita yung tvc na para kay binay hehe
4992,SOBRA ?????? Puro sila Roxas samantalang pag k...,1,sobra puro sila roxas samantalang pag kay dut...
4993,ElectionHugot:“We lose the presidency not once...,0,electionhugotwe lose the presidency not once b...
4994,@jefk_rew pity those poor people who were invo...,1,pity those poor people who were involved in th...
4995,Awww DUTERTE Na wag Lang si Roxas,0,awww duterte na wag lang si roxas
4996,RT @mikkieugenio: If the SC disqualifies Poe a...,0,if the sc disqualifies poe as president eithe...
4997,"""Pag naging presidente si Binay, wala kayong t...",0,pag naging presidente si binay wala kayong tax...
4998,Yan na naman ang walang kwentang commercial ni...,1,yan na naman ang walang kwentang commercial ni...


In [24]:
first_column = train.pop('label')
train.insert(0,'label',first_column)

In [25]:
train.head(10)

,label,text,clean_tweet
0,0,Inaasahan na ni Vice President Jejomar Binay n...,inaasahan na ni vice president jejomar binay n...
1,1,Mar Roxas TANG INA TUWID NA DAAN DAW .. EH SYA...,mar roxas tang ina tuwid na daan daw eh sya n...
2,0,Salamat sa walang sawang suporta ng mga taga m...,salamat sa walang sawang suporta ng mga taga m...
3,1,@rapplerdotcom putangina mo binay TAKBO PA,putangina mo binay takbo pa
4,0,"Binay with selective amnesia, forgetting about...",binay with selective amnesia forgetting about ...
5,0,It doesn't matter whoever won between Duterte ...,it doesnt matter whoever won between duterte &...
6,1,Nognog? Pero nognog din ang nag malasakit? Wtf...,nognog pero nognog din ang nag malasakit wtf t...
8,0,What Abi Binay said on running for Makati mayo...,what abi binay said on running for makati mayor
9,1,Srsly. How can Binay do away with no tax for t...,srsly how can binay do away with no tax for th...
10,1,"Sen Grace Poe, puro ka puso. Kaya lugmok bansa...",sen grace poe puro ka puso kaya lugmok bansa n...


In [26]:
#total data entries for training

print(train.shape)

(4747, 3)


In [27]:
# remove unwanted text from test data
test.head(10)

,text,label
0,Unshaded votes and votes for Mayor Duterte goe...,1
1,#NoMoreChance https://t.co/msaaUGv0bS,1
2,@itsmanj well there's other good choices like ...,0
3,Nognog. Pandak. Laki sa hirap. Pero corrupt. Y...,1
4,Ex-Binay aide turns tables on Mercado | https:...,0
5,2015/10/15 11:37 PHT,0
6,"Binay in Isabela radio interview: ‘Yong iba, ...",0
7,Poor Binay ?? https://t.co/In9yRo5QXI,1
8,RT @sizzlingbubbles: Juskopo! Gugustuhin kong ...,1
9,"ISA LANG MASASABI KO SA AD NI BINAY, P U T A. ...",1


In [28]:
first_column = test.pop('label')
test.insert(0,'label',first_column)

In [29]:
test.head(10)

,label,text
0,1,Unshaded votes and votes for Mayor Duterte goe...
1,1,#NoMoreChance https://t.co/msaaUGv0bS
2,0,@itsmanj well there's other good choices like ...
3,1,Nognog. Pandak. Laki sa hirap. Pero corrupt. Y...
4,0,Ex-Binay aide turns tables on Mercado | https:...
5,0,2015/10/15 11:37 PHT
6,0,"Binay in Isabela radio interview: ‘Yong iba, ..."
7,1,Poor Binay ?? https://t.co/In9yRo5QXI
8,1,RT @sizzlingbubbles: Juskopo! Gugustuhin kong ...
9,1,"ISA LANG MASASABI KO SA AD NI BINAY, P U T A. ..."


In [30]:
test_tweet = clean_tweets(test["text"])
test_tweet = pd.DataFrame(test_tweet)

In [31]:
# append cleaned tweets to the training data
test["clean_tweet"] = test_tweet

# compare the cleaned and uncleaned tweets
test.head(10)

,label,text,clean_tweet
0,1,Unshaded votes and votes for Mayor Duterte goe...,unshaded votes and votes for mayor duterte goe...
1,1,#NoMoreChance https://t.co/msaaUGv0bS,
2,0,@itsmanj well there's other good choices like ...,well theres other good choices like duterte or...
3,1,Nognog. Pandak. Laki sa hirap. Pero corrupt. Y...,nognog pandak laki sa hirap pero corrupt yan s...
4,0,Ex-Binay aide turns tables on Mercado | https:...,ex binay aide turns tables on mercado
5,0,2015/10/15 11:37 PHT,10 15 37 pht
6,0,"Binay in Isabela radio interview: ‘Yong iba, ...",binay in isabela radio interview yong iba nang...
7,1,Poor Binay ?? https://t.co/In9yRo5QXI,poor binay
8,1,RT @sizzlingbubbles: Juskopo! Gugustuhin kong ...,juskopo gugustuhin kong ma clark ma leah pwed...
9,1,"ISA LANG MASASABI KO SA AD NI BINAY, P U T A. ...",isa lang masasabi ko sa ad ni binay p u t a bo...


In [32]:
#https://statisticsglobe.com/drop-rows-blank-values-from-pandas-dataframe-python
test['clean_tweet'] = test['clean_tweet'].replace('', float('NaN'), regex = True)
test.dropna(inplace= True)


In [33]:
test.head(10)

,label,text,clean_tweet
0,1,Unshaded votes and votes for Mayor Duterte goe...,unshaded votes and votes for mayor duterte goe...
2,0,@itsmanj well there's other good choices like ...,well theres other good choices like duterte or...
3,1,Nognog. Pandak. Laki sa hirap. Pero corrupt. Y...,nognog pandak laki sa hirap pero corrupt yan s...
4,0,Ex-Binay aide turns tables on Mercado | https:...,ex binay aide turns tables on mercado
5,0,2015/10/15 11:37 PHT,10 15 37 pht
6,0,"Binay in Isabela radio interview: ‘Yong iba, ...",binay in isabela radio interview yong iba nang...
7,1,Poor Binay ?? https://t.co/In9yRo5QXI,poor binay
8,1,RT @sizzlingbubbles: Juskopo! Gugustuhin kong ...,juskopo gugustuhin kong ma clark ma leah pwed...
9,1,"ISA LANG MASASABI KO SA AD NI BINAY, P U T A. ...",isa lang masasabi ko sa ad ni binay p u t a bo...
10,0,#OnlyBinayWinner Only B1NAY,only b1nay


In [34]:
#remove stopwords
import urllib.request, json 
with urllib.request.urlopen("https://raw.githubusercontent.com/stopwords-iso/stopwords-tl/master/stopwords-tl.json") as url:
    stopwords = json.loads(url.read().decode())
    print(stopwords)

['akin', 'aking', 'ako', 'alin', 'am', 'amin', 'aming', 'ang', 'ano', 'anumang', 'apat', 'at', 'atin', 'ating', 'ay', 'bababa', 'bago', 'bakit', 'bawat', 'bilang', 'dahil', 'dalawa', 'dapat', 'din', 'dito', 'doon', 'gagawin', 'gayunman', 'ginagawa', 'ginawa', 'ginawang', 'gumawa', 'gusto', 'habang', 'hanggang', 'hindi', 'huwag', 'iba', 'ibaba', 'ibabaw', 'ibig', 'ikaw', 'ilagay', 'ilalim', 'ilan', 'inyong', 'isa', 'isang', 'itaas', 'ito', 'iyo', 'iyon', 'iyong', 'ka', 'kahit', 'kailangan', 'kailanman', 'kami', 'kanila', 'kanilang', 'kanino', 'kanya', 'kanyang', 'kapag', 'kapwa', 'karamihan', 'katiyakan', 'katulad', 'kaya', 'kaysa', 'ko', 'kong', 'kulang', 'kumuha', 'kung', 'laban', 'lahat', 'lamang', 'likod', 'lima', 'maaari', 'maaaring', 'maging', 'mahusay', 'makita', 'marami', 'marapat', 'masyado', 'may', 'mayroon', 'mga', 'minsan', 'mismo', 'mula', 'muli', 'na', 'nabanggit', 'naging', 'nagkaroon', 'nais', 'nakita', 'namin', 'napaka', 'narito', 'nasaan', 'ng', 'ngayon', 'ni', 'nila',

In [35]:
#data['content2'] =data['Content'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stopwords)]))
train['rm_stpwrds'] = train['clean_tweet'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stopwords) ]))

In [36]:
train[['clean_tweet', 'rm_stpwrds']].head(10)

,clean_tweet,rm_stpwrds
0,inaasahan na ni vice president jejomar binay n...,inaasahan vice president jejomar binay taong
1,mar roxas tang ina tuwid na daan daw eh sya n...,mar roxas tang ina tuwid daan daw eh sya nga d...
2,salamat sa walang sawang suporta ng mga taga m...,salamat sawang suporta taga makati pagbabalik ...
3,putangina mo binay takbo pa,putangina mo binay takbo
4,binay with selective amnesia forgetting about ...,binay with selective amnesia forgetting about ...
5,it doesnt matter whoever won between duterte &...,it doesnt matter whoever won between duterte &...
6,nognog pero nognog din ang nag malasakit wtf t...,nognog nognog nag malasakit wtf tangina mo bin...
8,what abi binay said on running for makati mayor,what abi binay said on running for makati mayor
9,srsly how can binay do away with no tax for th...,srsly how can binay do away with no tax for th...
10,sen grace poe puro ka puso kaya lugmok bansa n...,sen grace poe puro puso lugmok bansa natin kah...


In [37]:
#tokenization 

import nltk
from nltk.tokenize import WhitespaceTokenizer
train['tokenize'] = train['rm_stpwrds'].apply(nltk.tokenize.WhitespaceTokenizer().tokenize) 
train['tokenize'].head()



0    [inaasahan, vice, president, jejomar, binay, t...
1    [mar, roxas, tang, ina, tuwid, daan, daw, eh, ...
2    [salamat, sawang, suporta, taga, makati, pagba...
3                        [putangina, mo, binay, takbo]
4    [binay, with, selective, amnesia, forgetting, ...
Name: tokenize, dtype: object

In [38]:
train[['rm_stpwrds','tokenize']].head(30)

,rm_stpwrds,tokenize
0,inaasahan vice president jejomar binay taong,"[inaasahan, vice, president, jejomar, binay, t..."
1,mar roxas tang ina tuwid daan daw eh sya nga d...,"[mar, roxas, tang, ina, tuwid, daan, daw, eh, ..."
2,salamat sawang suporta taga makati pagbabalik ...,"[salamat, sawang, suporta, taga, makati, pagba..."
3,putangina mo binay takbo,"[putangina, mo, binay, takbo]"
4,binay with selective amnesia forgetting about ...,"[binay, with, selective, amnesia, forgetting, ..."
5,it doesnt matter whoever won between duterte &...,"[it, doesnt, matter, whoever, won, between, du..."
6,nognog nognog nag malasakit wtf tangina mo bin...,"[nognog, nognog, nag, malasakit, wtf, tangina,..."
8,what abi binay said on running for makati mayor,"[what, abi, binay, said, on, running, for, mak..."
9,srsly how can binay do away with no tax for th...,"[srsly, how, can, binay, do, away, with, no, t..."
10,sen grace poe puro puso lugmok bansa natin kah...,"[sen, grace, poe, puro, puso, lugmok, bansa, n..."


In [39]:
# POS tags 
# english_tags = ['NN', 'NNS', 'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ']
# tagalog_tags = ['NN', 'NNS', 'VB', 'VBP', 'JJ', 'RB']

# # detect the language 
# for i, tokens in enumerate(train['tokenize']):
#      if i >= 50:
#          break
#      tags = nltk.pos_tag(tokens)
#      english_count = sum(1 for tag in tags if tag[1] in english_tags)
#      tagalog_count = sum(1 for tag in tags if tag[1] in tagalog_tags)
#      if english_count > tagalog_count:
#          print("English:", ' '.join(tokens))
#      elif english_count < tagalog_count:
#          print("Tagalog:", ' '.join(tokens))
#      else:
#          print("Taglish:", ' '.join(tokens))

#comment ko muna di ko kasi mahanap dependencies hehe

# Proceeding to Training

In [40]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(train['tokenize'], train['label'], test_size=0.2, random_state=1)


In [41]:
# Vectorize the sentences using TfidfVectorizer
vectorizer = TfidfVectorizer()
X_train = vectorizer.fit_transform(map(' '.join, X_train))
X_test = vectorizer.transform(map(' '.join, X_test))

In [42]:
# Train the SVM model
clf = svm.SVC()
clf.fit(X_train, y_train)

# Test the model on the testing set
y_pred = clf.predict(X_test)

# Evaluate the model's performance
accuracy = clf.score(X_test, y_test)
print("Accuracy:", accuracy)

Accuracy: 0.7263157894736842


In [43]:
import re

# Replace digits with an empty string
test['clean_tweet'] = test['clean_tweet'].replace(to_replace=r'\d', value='', regex=True)

# Remove any remaining whitespace
test['clean_tweet'] = test['clean_tweet'].apply(lambda x: re.sub('\s+', ' ', x).strip())


train['clean_tweet'].to_csv('tokens.txt', header=None, index=None, sep=' ')


In [44]:
import sys
from langdetect import detect
VOWELS = "aeiouAEIOU"
CONSONANTS = "bcdfghklmnngpqrstvwyBCDFGHKLMNNGPQRSTVWY"

PREFIX_SET = [
	'nakikipag', 'pakikipag',
	'pinakama', 'pagpapa',
	'pinagka', 'panganga', 
	'makapag', 'nakapag', 
	'tagapag', 'makipag', 
	'nakipag', 'tigapag',
	'pakiki', 'magpa',
	'napaka', 'pinaka',
	'ipinag', 'pagka', 
	'pinag', 'mapag', 
	'mapa', 'taga', 
	'ipag', 'tiga', 
	'pala', 'pina', 
	'pang', 'naka',
	'nang', 'mang',
	'sing',
	'ipa', 'pam',
	'pan', 'pag',
	'tag', 'mai',
	'mag', 'nam',
	'nag', 'man',
	'may', 'ma',
	'na', 'ni',
	'pa', 'ka',
	'um', 'in',
	'i',
]

INFIX_SET = [
	'um', 'in',
]

SUFFIX_SET = [
	'syon','dor', 
	'ita', 'han', 
	'hin', 'ing', 
	'ang', 'ng', 
	'an', 'in', 
	'g',
]


PERIOD_FLAG = True
PASS_FLAG = False

def stemmer(source):
    global PERIOD_FLAG
    global PASS_FLAG

    word_info = {}
    stemmed = []
    word_root = []
    root_only = []
    errors = []
    pre_stem = inf_stem = suf_stem = rep_stem = \
        du1_stem = du2_stem = cle_stem = '-'

    PREFIX = []
    INFIX = []
    SUFFIX = []
    DUPLICATE = []
    REPITITION = []
    CLEANERS = []

    tokens = read_file(source)

    for token in tokens:
        word_info["word"] = token

		
        if (PERIOD_FLAG == True and token[0].isupper()) or \
                (PERIOD_FLAG == False and token[0].islower()):

            token = token.lower()
            du1_stem = clean_duplication(token, DUPLICATE)
            pre_stem = clean_prefix(du1_stem, PREFIX)
            rep_stem = clean_repitition(pre_stem, REPITITION)
            inf_stem = clean_infix(rep_stem, INFIX)
            rep_stem = clean_repitition(inf_stem, REPITITION)
            suf_stem = clean_suffix(rep_stem, SUFFIX)
            du2_stem = clean_duplication(suf_stem, DUPLICATE)
            cle_stem = clean_stemmed(du2_stem, CLEANERS, REPITITION)
            cle_stem = clean_duplication(cle_stem, DUPLICATE)

            if '-' in cle_stem:
                cle_stem.replace('-', '')

            if check_validation(cle_stem) == False:
                PASS_FLAG = True
                du1_stem = clean_duplication(cle_stem, DUPLICATE)
                pre_stem = clean_prefix(du1_stem, PREFIX)
                rep_stem = clean_repitition(pre_stem, REPITITION)
                inf_stem = clean_infix(rep_stem, INFIX)
                rep_stem = clean_repitition(inf_stem, REPITITION)
                suf_stem = clean_suffix(rep_stem, SUFFIX)
                du2_stem = clean_duplication(suf_stem, DUPLICATE)
                cle_stem = clean_stemmed(du2_stem, CLEANERS, REPITITION)
                cle_stem = clean_duplication(cle_stem, DUPLICATE)

            word_info["root"] = cle_stem

            PASS_FLAG = False
            PERIOD_FLAG = False
            PREFIX = []
            INFIX = []
            SUFFIX = []
            DUPLICATE = []
            REPITITION = []
            CLEANERS = []

        else:
            PERIOD_FLAG = False
            cle_stem = clean_stemmed(token, CLEANERS, REPITITION)
            word_info["root"] = token

        stemmed.append(word_info)
        root_only.append(word_info["root"])
        word_root.append(word_info["word"] + ' : ' + word_info["root"])

        print(token + ' : ' + word_info["root"])

        word_info = {}
        pre_stem = inf_stem = suf_stem = rep_stem = \
            du1_stem = du2_stem = cle_stem = '-'

    return stemmed, root_only


def clean_duplication(token, DUPLICATE):

	if check_validation(token):
		return token

	if '-' in token and token.index('-') != 0 and \
		token.index('-') != len(token) -  1:

		split = token.split('-')

		if all(len(tok) >= 3 for tok in split):
			if split[0] == token[1] or split[0][-1] == 'u' and change_letter(split[0], -1, 'o') == split[1] or \
				split[0][-2] == 'u' and change_letter(split[0], -2, 'o')  == split[1]:
				DUPLICATE.append(split[0])
				return split[0]

			elif split[0] == split[1][0:len(split[0])]:
				DUPLICATE.append(split[1])
				return split[1]

			elif split[0][-2:] == 'ng':
				if split[0][-3] == 'u':
					if split[0][0:-3] + 'o' == split[1]:
						DUPLICATE.append(split[1])
						return split[1]

				if split[0][0:-2] == split[1]:
					DUPLICATE.append(split[1])
					return split[1]

		else:
			return '-'.join(split)
	
	return token


def clean_repitition(token, REPITITION):
	"""
		Checks token for repitition. (ex. nakakabaliw = nabaliw)
			token: word to be stemmed repitition
		returns STRING
	"""

	if check_validation(token):
		return token

	if len(token) >= 4:
		if check_vowel(token[0]):
			if token[0] == token[1]:
				REPITITION.append(token[0])
				return token[1:]

		elif check_consonant(token[0]) and count_vowel(token) >= 2:
			if token[0: 2] == token[2: 4] and len(token) - 2 >= 4:
				REPITITION.append(token[2:4])
				return token[2:]
			
			elif token[0: 3] == token[3: 6] and len(token) - 3 >= 4:
				REPITITION.append(token[3:6])
				return token[3:]

	return token


def clean_prefix(token,	 PREFIX):

	if check_validation(token):
		return token

	for prefix in PREFIX_SET:
		if len(token) - len(prefix) >= 3 and \
			count_vowel(token[len(prefix):]) >= 2:

			if prefix == ('i') and check_consonant(token[2]):
				continue

			if '-' in token:	
				token = token.split('-')

				if token[0] == prefix and check_vowel(token[1][0]):
					PREFIX.append(prefix)
					return token[1]

				token = '-'.join(token)

			if token[0: len(prefix)] == prefix:
				if count_vowel(token[len(prefix):]) >= 2:
					# if check_vowel(token[len(token) - len(prefix) - 1]):
				# 	continue

					if prefix == 'panganga':
						PREFIX.append(prefix)
						return 'ka' + token[len(prefix):]
					
					PREFIX.append(prefix)
					return token[len(prefix):]

	return token

 
def clean_infix(token, INFIX):
	"""
		Checks token for infixes. (ex. bumalik = balik)
			token: word to be stemmed for infixes
		returns STRING
	"""

	if check_validation(token):
		return token

	for infix in INFIX_SET:
		if len(token) - len(infix) >= 3 and count_vowel(token[len(infix):]) >= 2:
			if token[0] == token[4] and token[1: 4] == infix:
				INFIX.append(infix)
				return token[4:]

			elif token[2] == token[4] and token[1: 3] == infix:
				INFIX.append(infix)
				return token[0] + token[3:]

			elif token[1: 3] == infix and check_vowel(token[3]):
				INFIX.append(infix)
				return token[0] + token[3:]

	return token


def clean_suffix(token, SUFFIX):
	"""
		Checks token for suffixes. (ex. bigayan = bigay)
			token: word to be stemmed for suffixes
		returns STRING
	"""

	SUF_CANDIDATE = []

	if check_validation(token):
		return token
	
	for suffix in SUFFIX_SET:
		if len(token) - len(suffix) >= 3 and count_vowel(token[0:len(token) - len(suffix)]) >= 2:
			if token[len(token) - len(suffix): len(token)] == suffix:
				if len(suffix) == 2 and not count_consonant(token[0:len(token) - len(suffix)]) >= 1:
					continue

				if count_vowel(token[0: len(token) - len(suffix)]) >= 2:
					if suffix == 'ang' and check_consonant(token[-4]) \
						and token[-4] != 'r' and token[-5] != 'u':
						continue

					# if check_vowel(suffix[0]) and check_consonant(token[len])

					print(token[0: len(token) - len(suffix)] + " : " + suffix)

					if check_validation(token[0: len(token) - len(suffix)]):
						SUFFIX.append(suffix)
						return token[0: len(token) - len(suffix)] + 'a' if SUFFIX == 'ita' \
							else  token[0: len(token) - len(suffix)]

					elif len(SUF_CANDIDATE) == 0:
						SUF_CANDIDATE.append(suffix)
						SUF_CANDIDATE.append(token[0: len(token) - len(suffix)])

	if(len(SUF_CANDIDATE) == 2):
		SUFFIX = SUF_CANDIDATE[0]
		return SUF_CANDIDATE[1][0: len(token) - len(suffix)] + 'a' if SUFFIX == 'ita' \
			else  SUF_CANDIDATE[1][0: len(token) - len(suffix)]

	return token


def check_vowel(substring):
	"""
		Checks if the substring is a vowel.
			letters: substring to be tested
		returns BOOLEAN
	"""

	return all(letter in VOWELS for letter in substring)


def check_consonant(substring):
	"""
		Checks if the letter is a consonant.
			letter: substring to be tested
		returns BOOLEAN
	"""

	return all(letter in CONSONANTS for letter in substring)



def count_vowel(token):
	"""
		Count vowels in a given token.
			token: string to be counted for vowels
		returns INTEGER
	"""

	count = 0

	for tok in token:
		if check_vowel(tok):
			count+=1

	return count


def count_consonant(token):
	"""
		Count consonants in a given token.
			token: string to be counted for consonants
		returns INTEGER
	"""

	count = 0

	for tok in token:
		if check_consonant(tok):
			count+=1

	return count


def change_letter(token, index, letter):
	"""
		Replaces a letter in a token.
			token: word to be used
			index: index of the letter
			letter: letter used to replace
		returns STRING
	"""
	
	_list = list(token)
	_list[index] = letter

	return ''.join(_list)


def clean_stemmed(token, CLEANERS, REPITITION):
	"""
		Checks for left-over affixes and letters.
			token: word to be cleaned for excess affixes/letters
		returns STRING
	"""

	global PERIOD_FLAG
	global PASS_FLAG

	CC_EXP = ['dr', 'gl', 'gr', 'ng', 'kr', 'kl', 'kw', 'ts', 'tr', 'pr', 'pl', 'pw', 'sw', 'sy'] # Consonant + Consonant Exceptions

	if token[-1] == '.' and PASS_FLAG == False:
		PERIOD_FLAG = True

	if not check_vowel(token[-1]) and not check_consonant(token[-1]):
		CLEANERS.append(token[-1])
		token = token[0:-1]

	if not check_vowel(token[0]) and not check_consonant(token[0]):
		CLEANERS.append(token[0])
		token = token[1:]

	if check_validation(token):
		return token

	if len(token) >= 3 and count_vowel(token) >= 2:
		token = clean_repitition(token,	REPITITION)

		if check_consonant(token[-1]) and token[- 2] == 'u':
			CLEANERS.append('u')
			token = change_letter(token, -2, 'o')

		if token[len(token) - 1] == 'u':
			CLEANERS.append('u')
			token = change_letter(token, -1, 'o')

		if token[-1] == 'r':
			CLEANERS.append('r')
			token = change_letter(token, -1, 'd')

		if token[-1] == 'h' and check_vowel(token[-1]):
			CLEANERS.append('h')
			token = token[0:-1]

		# if token[0] == 'i':
		# 	token = token[1:]

		if token[0] == token[1]:
			CLEANERS.append(token[0])
			token = token[1:]

		if (token[0: 2] == 'ka' or token[0: 2] == 'pa') and check_consonant(token[2]) \
			and count_vowel(token) >= 3:
			
			CLEANERS.append(token[0: 2])
			token = token[2:]

		if(token[-3:]) == 'han' and count_vowel(token[0:-3]) == 1:
			CLEANERS.append('han')
			token = token[0:-3] + 'i'

		if(token[-3:]) == 'han' and count_vowel(token[0:-3]) > 1:
			CLEANERS.append('han')
			token = token[0:-3]

		if len(token) >= 2 and count_vowel(token) >= 3:
			if token[-1] == 'h' and check_vowel(token[-2]):
				CLEANERS.append('h')
				token = token[0:-1]

		if len(token) >= 6 and token[0:2] == token[2:4]:
			CLEANERS.append('0:2')
			token = token[2:]

		if any(REP[0] == 'r' for REP in REPITITION):
			CLEANERS.append('r')
			token = change_letter(token, 0, 'd')

		if token[-2:] == 'ng' and token[-3] == 'u':
			CLEANERS.append('u')
			token = change_letter(token, -3, 'o')

		if token[-1] == 'h':
			CLEANERS.append('h')
			token = token[0:-1]

		if any(token[0:2] != CC for CC in CC_EXP) and check_consonant(token[0:2]):
			CLEANERS.append(token[0:2])
			token = token[1:]

	return token


def read_file(source):
	"""
		Gets content of a text file.
			source: file name
		returns LIST
	"""

	with open(source, 'r') as myfile:
		data = myfile.read().replace('\n', ' ')

	return data.split(' ')


def write_file(stemmed_info, word_root, root):
	"""
		Creates a log for the output.
			stemmed_info: list of dicts with stemming info
			word_root: format [word : root]
			root: list of stemmed words
		returns NULL
	"""

	with open('output/with_info.txt', 'w') as with_info, \
		open('output/root_word.txt', 'w') as root_word, \
		open('output/root_only.txt', 'w') as root_only:
		
		for inf, rw, ro in zip(stemmed_info, word_root, root):
			with_info.write(str(inf) + '\n')
			root_word.write(rw + '\n')
			root_only.write(ro + '\n')


def check_validation(token):
	with open('stemmer/validation.txt', 'r') as valid:
		data = valid.read().replace('\n', ' ').split(' ')
		# data = set([line.strip('\n') for line in valid.readlines()])

	return True if token in data else False



def validate(stemmed, errors):
	"""
		Calculates accuracy.
			stemmed: list of stemmed words
			errors: list of stemming errors
		returns FLOAT
	"""

	check = 0

	with open('stemmer/validation.txt', 'r') as valid:
		data = valid.read().replace('\n', ' ').split(' ')
		# data = set([line.strip('\n') for line in valid.readlines()])
		
	for stem in stemmed:
		if stem[0].isupper() or stem in data:
			check += 1

		else:
			errors.append(stem)
	
	return format((check / len(stemmed) * 100), '.2f')

ModuleNotFoundError: No module named 'langdetect'

In [ ]:
import csv

# open the CSV file
with open('output3.csv', 'r') as csv_file:
    csv_reader = csv.reader(csv_file)

    # open the output text file
    with open('clean.txt', 'w') as txt_file:
        # loop through each row of the CSV file and write it to the text file
        for row in csv_reader:
            txt_file.write(','.join(row) + '\n')


In [ ]:
with open("clean.txt", "r") as input_file, open("cleaner.txt", "w") as output_file:
    for line in input_file:
        updated_line = line.replace("&", "").replace("*", "").replace("_", "").replace("+", "").replace("=", "").replace("~", "")
        output_file.write(updated_line)


In [ ]:
stemmer('cleaner.txt')

unshaded : unshaded
vote : vote
and : and
vote : vote
for : for
mayor : mayod
duterte : duterte
go : go
to : to
mar : mar
roxas : roxas
accord : accord
to : to
some : some
report : report
of : of
ballot : ballot
test : test
well : well
there : here
other : othed
good : good
choice : hoice
like : like
duterte : duterte
or : or
poe : poe
but : but
both : both
of : of
them : them
be : be
still : still
undecided : undecided
i : i
think : think
nogno : g
nognog : nogno
pandak : pandak
laki : laki
sa : sa
hirap : hirap
pero : pero
corrupt : corrupt
yan : yan
si : si
binay : binay
ex : e
binay : binay
aide : aide
turn : turn
table : table
on : on
mercado : mercado
pht : pht
binay : binay
in : in
isabela : sabela
radio : radio
interview : terview
yong : yong
iba : iba
nangangako : angako
pa : pa
nagawa : awa
ko : ko
na : na
ho : ho
lahat : lahat
yon : yon
poor : pood
binay : binay
juskopo : uskopo
gustu : hin
gustuh : in
gugustuhin : gusto
kong : kong
ma : ma
clark : clark
ma : ma
leah : lea
p

IndexError: string index out of range

In [ ]:
"""
Tagalog Stemmer
"""

VOWELS = "aeiouAEIOU"
CONSONANTS = "bcdfghklmnngpqrstvwyBCDFGHKLMNNGPQRSTVWY"

PREFIX_SET = [
    'nakikipag', 'pakikipag',
    'pinakama', 'pagpapa',
    'pinagka', 'panganga', 
    'makapag', 'nakapag', 
    'tagapag', 'makipag', 
    'nakipag', 'tigapag',
    'pakiki', 'magpa',
    'napaka', 'pinaka',
    'ipinag', 'pagka', 
    'pinag', 'mapag', 
    'mapa', 'taga', 
    'ipag', 'tiga', 
    'pala', 'pina', 
    'pang', 'naka',
    'nang', 'mang',
    'sing',
    'ipa', 'pam',
    'pan', 'pag',
    'tag', 'mai',
    'mag', 'nam',
    'nag', 'man',
    'may', 'ma',
    'na', 'ni',
    'pa', 'ka',
    'um', 'in',
    'i',
]

INFIX_SET = [
    'um', 'in',
]

SUFFIX_SET = [
    'syon','dor', 
    'ita', 'han', 
    'hin', 'ing', 
    'ang', 'ng', 
    'an', 'in', 
    'g',
]


def tagalog_stemmer(word):
    """
    This function takes a Tagalog word and returns its root form by removing affixes.
    """

    # Remove repeating syllables
    for i in range(len(word) - 1):
        if word[i] == word[i+1] and word[i] in VOWELS:
            word = word[:i] + word[i+1:]

    # Remove prefix
    for prefix in PREFIX_SET:
        if word.startswith(prefix):
            word = word[len(prefix):]
            break

    # Remove suffix
    for suffix in SUFFIX_SET:
        if word.endswith(suffix):
            word = word[:-len(suffix)]
            break

    # Remove infix
    for infix in INFIX_SET:
        if infix in word:
            parts = word.split(infix)
            # Check if the infix is surrounded by two consonants or a vowel and a consonant
            if len(parts) == 2 and \
                all(c in CONSONANTS for c in parts[0]) and \
                all(c in CONSONANTS for c in parts[1][1:]) and \
                parts[1][0] in VOWELS and \
                len(parts[0]) > 0 and len(parts[1]) > 1:
                word = parts[0] + parts[1][1:]
                break
    


    return word

word = "tatakbo"
root_word = tagalog_stemmer(word)
print(root_word) 


tatakbo


In [ ]:
import csv
from langdetect import detect

# Open CSV file
with open('test.csv', 'r', encoding='utf-8') as csvfile:
    reader = csv.reader(csvfile)

    # Loop over rows
    count = 0
    for row in reader:
        # Check if 30 sentences have been printed
        if count >= 30:
            break

        sentence = row[0]
        try:
            # Try to detect language
            language = detect(sentence)
        except:
            # If langdetect fails, skip sentence
            continue

        # Print sentence and detected language
        if language == 'en':
            print(f'English: {sentence}')
        elif language == 'tl':
            print(f'Tagalog: {sentence}')

        count += 1


FileNotFoundError: [Errno 2] No such file or directory: 'test.csv'

In [ ]:
test[['clean_tweet']].to_csv('clean_tweet.csv', index=False)


In [ ]:
import csv
import re
from nltk.stem import PorterStemmer
from langdetect import detect

csv_file_path = "clean_tweet.csv" 
output_file_path = "output.csv" 

def word_stemmer(word):

    stemmer = PorterStemmer()

    stemmed_word = stemmer.stem(stemmer.stem(word)) 

    return stemmed_word


with open(csv_file_path, "r") as csv_file:
    reader = csv.reader(csv_file)
    with open(output_file_path, "w", newline='') as output_file:
        writer = csv.writer(output_file)
        for row in reader:
            sentence = row[1]  
            words = sentence.split()  
            stemmed_sentence = []
            for word in words:
                # Skip numbers and non-English words
                if re.match(r'^[a-zA-Z]+$', word) and detect(word) == 'en':
                    stemmed_sentence.append(word_stemmer(word))
                else:
                    stemmed_sentence.append(word)
            stemmed_sentence = " ".join(stemmed_sentence) 
            writer.writerow([stemmed_sentence]) 



In [ ]:
import csv
import nltk
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize

def is_english_word(word):
    return wordnet.synsets(word, lang='eng')

def lemmatize(text):
    lemmatizer = nltk.WordNetLemmatizer()
    tokens = word_tokenize(text)
    lemmatized_words = []
    for token in tokens:
        # Skip numbers and non-English words
        if token.isdigit() or not is_english_word(token):
            lemmatized_words.append(token)
            continue
        lemma = lemmatizer.lemmatize(token, wordnet.VERB) # You can specify different parts-of-speech (POS) tags to get the desired lemmatization
        if lemma == token:
            lemma = lemmatizer.lemmatize(token, wordnet.NOUN)
        lemmatized_words.append(lemma)
    lemmatized_text = ' '.join(lemmatized_words)
    return lemmatized_text

# Load CSV file
with open('clean_tweet.csv', 'r') as file:
    reader = csv.reader(file)
    header = next(reader)  

    lemmatized_sentences = []
    for row in reader:
        sentence = row[0]  
        lemmatized_sentence = lemmatize(sentence)
        lemmatized_sentences.append([lemmatized_sentence])

# Save lemmatized output to a CSV file
with open('output3.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerows(lemmatized_sentences)
